In [1]:
import json
import numpy as np
import os
import random
import pandas as pd
import pickle
import seaborn as sns
import typing 

from IPython.display import clear_output
from misc import _get_usleep_token
from matplotlib import pyplot as plt
from matplotlib.patches import Polygon
from pprint import pprint
from scipy import io, special, stats
from sklearn import metrics, model_selection
from statistics import mode
from tqdm import tqdm
from usleep_api import USleepAPI

from utils import *

In [2]:
# GLOBALS 

MS_MAPPING = {"Wake": 0, "MS": 1}
AASM_MAPPING = {"Wake": 0, "N1": 1, "N2": 2, "N3": 3, "REM": 4}

float_formatter = "{:.2f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})

%matplotlib widget

## Subset data
Select a subset (n = 5) of data to use for pre-liminary analysis of microsleep predictions using a fully trained U-Sleep model.
Set the seed to 42

In [4]:
all_edf_files = np.array(sorted(os.listdir("edf_data/")))
all_labels_files = np.array(sorted(os.listdir("labels/")))
all_names = [x.replace(".edf","") for x in all_edf_files]
assert [x.replace(".edf", "") for x in all_edf_files] == [x.replace(".mat","") for x in all_labels_files]

# use_bern_splits = True

# if use_bern_splits:
#     with open("skorucack_splits.json","r") as f:
#         splits = json.loads(f.read())

#     train_names = np.array(splits['train'])
#     train_idx = np.isin(all_names, train_names)
#     test_names = np.array(splits['test'])
#     test_idx = np.isin(all_names, test_names)
#     test_edf_files = all_edf_files[test_idx]
#     test_label_files = all_labels_files[test_idx]
# else:    
#     random.seed(42)
#     n_files = 5
#     train_idx = random.sample(range(0, len(all_edf_files)), n_files)


# edf_files = all_edf_files[train_idx]
# label_files = all_labels_files[train_idx]
# names = [x.replace(".edf", "") for x in edf_files]

## Preliminary analysis
* Run all pre-liminary recordings through 
    * U-Sleep API
    * ut command line
* Read and concatinate output files
* Plot

In [6]:
from ut_commands import predict_one
input_dir              = "edf_data"
output_dir             = "predictions"

out_files = []
# samples_per = [1,2,4,8,16]
samples_per = [128]
for s in samples_per:
    hz = int(128/s)
    for edf in all_edf_files:
        name = edf.replace(".edf","")
        inp_file = os.path.join(input_dir, edf)
        out_file = os.path.join(output_dir, f"{hz}_hz", f"{name}.npy")
        out_files.append(out_file)

        predict_one(input_file = inp_file, output_file=out_file, data_per_prediction=s)
        clear_output(wait=True)


ut predict_one -f edf_data\zaca.edf -o predictions\1_hz\zaca.npy --channels "O1-M2"==EEG "O2-M1"==EEG "E1-M1"==EOG "E2-M1"==EOG --auto_channel_grouping EOG EEG --model u-sleep:1.0 --logging_out_path logs\predictions\1_hz\zaca.log --num_gpus 0 --data_per_prediction 128 --no_argmax --overwrite


In [23]:


# Set some parameters
model                  = "U-Sleep v1.0"
input_dir              = "edf_data"
output_dir             = "predictions"
predictions_per_second = 8
channel_groups         = [['EEG O1-M2', 'EOG LOC-M1'],
                          ['EEG O2-M1', 'EOG LOC-M1'],
                          ['EEG O1-M2', 'EOG ROC-M1'],
                          ['EEG O2-M1', 'EOG ROC-M1']]
#try:
#    api = USleepAPI(api_token=os.environ["USLEEP_API_TOKEN"])
#except:
#    api = USleepAPI(api_token=_get_usleep_token())

out_files = []
pred_window = str(int(predictions_per_second**-1)) + "sec"
for edf in edf_files:
    
    name = edf.replace(".edf","")
    
    inp_file = os.path.join(input_dir, edf)
    out_file = os.path.join(output_dir, f"{predictions_per_second}_hz",f"{name}.npy")
    print(f"Running: {out_file}")
    
    #api.quick_predict(
    #     input_file_path=inp_file,
    #     model = model,
    #     output_file_path=out_file,
    #     data_per_prediction=128 * (1/predictions_per_second),
    #     channel_groups=channel_groups,
    #     with_confidence_scores=True
    # )

    out_files.append(out_file)


Running: predictions\8_hz\0ncr.npy
Running: predictions\8_hz\3J4W.npy
Running: predictions\8_hz\5bSg.npy
Running: predictions\8_hz\9098.npy
Running: predictions\8_hz\9JQY.npy
Running: predictions\8_hz\AXbm.npy
Running: predictions\8_hz\Af8a.npy
Running: predictions\8_hz\AsLD.npy
Running: predictions\8_hz\C1Wu.npy
Running: predictions\8_hz\DYYI.npy
Running: predictions\8_hz\DjrT.npy
Running: predictions\8_hz\Dr51.npy
Running: predictions\8_hz\EHED.npy
Running: predictions\8_hz\EMcQ.npy
Running: predictions\8_hz\G7PJ.npy
Running: predictions\8_hz\Ivfn.npy
Running: predictions\8_hz\JCpz.npy
Running: predictions\8_hz\LR2s.npy
Running: predictions\8_hz\MS6u.npy
Running: predictions\8_hz\Msy4.npy
Running: predictions\8_hz\Nzhl.npy
Running: predictions\8_hz\QDsa.npy
Running: predictions\8_hz\RM1S.npy
Running: predictions\8_hz\RfL0.npy
Running: predictions\8_hz\SOZ3.npy
Running: predictions\8_hz\UsSz.npy
Running: predictions\8_hz\UwK6.npy
Running: predictions\8_hz\X7s0.npy
Running: predictions

In [24]:

y_true = np.empty(shape=[0])
y_probs = np.empty(shape=[0,5])

mapping = {"Wake": [0, 2, 3], "MS": [1]}
uni_on = True
replace = False
pred_freq = 1

for idx, rec in enumerate(out_files):
    
    probs = get_probs(rec)
    probs_resampled = resample_usleep_preds(probs, data_per_pred = 1)#128 * (1/predictions_per_second))
    y_probs = np.concatenate([y_probs, probs_resampled])
    #y_probs = np.concatenate([y_probs, probs])
    
    if idx == 0:
        BERN = BernLabels(label_files[idx], mapping, uni_on)
        BERN.apply_time_critera(replace=replace)
    else:
        tmp = BernLabels(label_files[idx], mapping, uni_on)
        tmp.apply_time_critera(replace=replace)
        BERN.append(tmp)
    
    #assert BERN.num_labels == len(y_probs)
    clear_output(wait=True)
    
y_pred = np.argmax(y_probs,1)
y_pred = aasm_to_wake_sleep(y_pred)

# remove y_pred where outside MS definition
#_y_pred = remove_invalid_labels(y_pred, 1, fs=200)

y_true = BERN.labels

# Add predictions using cumulative sleep probabilites 
yy_probs = np.empty([y_probs.shape[0],2])
yy_probs[:,0] = y_probs[:,0]
yy_probs[:,1] = 1-y_probs[:,0]#np.sum(y_probs[:,1:],1)
yy_pred = np.argmax(yy_probs,1)
#_yy_pred = remove_invalid_labels(yy_pred, 1, fs=200)


0 labels are shorter than 3 seconds
2 labels are longer than 15 seconds



In [25]:
# Highest sleep class vs wake
y_bin = np.empty([y_probs.shape[0],2])
y_bin[:,0] = y_probs[:,0]
high_sleep_idx = np.argmax(y_probs[:,1:5],1) + 1
for val in np.unique(high_sleep_idx):
    idx = np.where(high_sleep_idx == val)
    y_bin[idx,1] = y_probs[idx,val]

yb_pred = np.argmax(y_bin,1)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1, figsize=[9,9], sharex=True)
lab_fs = 200
pred_fs = 1

if lab_fs == 200:
    xlab = "Time [s]"
else:
    xlab = "Sample #"
    
# Plot hypnodensity
plot_probs(ax1, y_probs, ["Wake", "N1", "N2", "N3", "REM"], fs = pred_fs)
ax1.autoscale(enable=True, axis='both', tight=True)

# Plot the predicted hypnogram
_,_=plot_label_patches(y_pred, {"Wake": 0, "MS": 1}, y_min=2/3, ax=ax2, fs= pred_fs)
_,_=plot_label_patches(yb_pred, {"Wake": 0, "MS": 1}, y_min=1/3, y_max=2/3, ax=ax2, fs = pred_fs)
hdl,names=plot_label_patches(yy_pred, {"Wake": 0, "MS": 1}, y_max=1/3, ax=ax2, fs= pred_fs)

ax2.legend(hdl, names, loc=1)
ax2.autoscale(enable=True, axis='both', tight=True)
ax2.plot([*ax2.get_xlim()],[1/3, 1/3],'-k',linewidth=0.5)
ax2.plot([*ax2.get_xlim()],[2/3, 2/3],'-k',linewidth=0.5)
ax2.set_yticks([0.5/3, 1.5/3, 2.5/3])
ax2.set_yticklabels(["yy_pred","yb_pred","y_pred"])

# Plot the true hypnogram
hdl, names = plot_label_patches(y_true, {"Wake": 0, "MS": 1}, ax = ax3, fs = lab_fs)
ax3.autoscale(enable=True, axis='both', tight=True)
ax3.legend(hdl, names, loc = 1)
ax3.set_xlabel(xlab)
clear_output(wait=False)
#plt.savefig(fig_path("overall"), format="png")